In [ ]:
import numpy as np

# -----------------------------------------------------
# SIGNAL MAPPING: convert raw signals → engineering units
# -----------------------------------------------------

def map_speed(raw):
    if raw is None:
        return None
    return raw * 0.1        # e.g., scale factor from ECU

def map_temperature(raw):
    if raw is None:
        return None
    return raw - 40         # example offset for sensor

def map_fuel(raw):
    if raw is None:
        return None
    return raw / 2          # half-scale sensor output

# Mapping functions dictionary
MAPPERS = {
    "speed": map_speed,
    "temperature": map_temperature,
    "fuel": map_fuel
}


# -----------------------------------------------------
# ASSUMPTION MODELING: handling missing or invalid data
# -----------------------------------------------------

DEFAULTS = {
    "speed": 0,
    "temperature": 25,
    "fuel": 50,
}

ASSUMPTIONS = {
    "speed": lambda x: 0 if x < 0 else x,         # negative = sensor failure
    "temperature": lambda x: min(max(x, -20), 120), # clamp between -20 and 120 C
    "fuel": lambda x: max(x, 0),                   # no negative fuel allowed
}


def apply_assumptions(signal_name, value):
    if value is None:
        return DEFAULTS[signal_name]  # fallback default

    # apply safety/law-of-physics rules
    return ASSUMPTIONS[signal_name](value)


# -----------------------------------------------------
# DERIVED SIGNAL MODELING (simple example)
# -----------------------------------------------------
def compute_derived_signals(mapped_signals):
    speed = mapped_signals["speed"]
    fuel = mapped_signals["fuel"]

    # Simple model: range estimate = fuel% × speed × factor
    range_estimate = fuel * speed * 0.2

    return {
        "range_estimate": round(range_estimate, 2)
    }


# -----------------------------------------------------
# PIPELINE
# -----------------------------------------------------
def process_signals(raw_signals):
    mapped = {}
    final = {}

    # 1) MAP RAW SIGNALS
    for key, value in raw_signals.items():
        mapped[key] = MAPPERS[key](value)

    # 2) APPLY ASSUMPTIONS
    for key, value in mapped.items():
        final[key] = apply_assumptions(key, value)

    # 3) DERIVED SIGNALS
    derived = compute_derived_signals(final)

    return {
        "mapped_signals": mapped,
        "final_signals": final,
        "derived_signals": derived
    }


# -----------------------------------------------------
# EXAMPLE RUN
# -----------------------------------------------------
raw_input = {
    "speed": 350,      # raw speed
    "temperature": None,  # missing signal
    "fuel": -10,       # bad signal
}

output = process_signals(raw_input)

print("\n=== Raw Input Signals ===")
print(raw_input)

print("\n=== Mapped Signals ===")
print(output["mapped_signals"])

print("\n=== Final (Assumption Applied) Signals ===")
print(output["final_signals"])

print("\n=== Derived Signals ===")
print(output["derived_signals"])


In [ ]:
speed = 350 → 35.0 km/h  
temperature = None → None  
fuel = -10 → -5  


In [ ]:
temperature = default 25 C  
fuel = clamp → 0  
speed = stays positive  


In [ ]:
range_estimate = fuel * speed * 0.2  
